In [2]:
ls

Conversations/
Twitter.zip
Twitter_Data.ipynb
Twitter_data/
extract_fet_review.py
hotel_review_preprocess.py
pan14-author-profiling-training-corpus-2014-04-16/
pan14-author-profiling-training-corpus-2014-04-16.zip
pan16-author-profiling-training-dataset-2016-04-25/
pan16-author-profiling-training-dataset-2016-04-25.zip
workArea.ipynb


In [1]:
cd Twitter_data/

/Users/aditya/Google Drive/UMass/First Semester/585/Project/WorkArea/Twitter_data


In [2]:
cd pan14-author-profiling-training-corpus-english-twitter-2014-04-16/

/Users/aditya/Google Drive/UMass/First Semester/585/Project/WorkArea/Twitter_data/pan14-author-profiling-training-corpus-english-twitter-2014-04-16


In [4]:
'''
Pre-Processing of the Text file, as per project proposal:
• Remove url to ‘url’, html tags, nums to ‘num’, @mentions
• Eliminate duplicate ‘retweet’ data, which adds undue bias to our dataset.
• Eliminate noisy tweets containing non-Latin characters.

PENDING:
• Use default svm param/ or maybe tune after finalizing features.
• Vary C after finalizing features.
'''
import regex
import nltk
import re
import string
from __future__ import division
import xml.etree.ElementTree as ET
import os
import nltk
import enchant

d1=enchant.Dict("en_US")

def preProcessing(line):
    #if line[0:3] == 'RT:':
    #   return False
    line = line.split()
    modifiedLine = []
    for i in range(len(line)):
        try:
            line[i].encode(encoding='utf-8').decode('ascii')
        except:
            continue
        
        if 'http' in line[i].lower() or 'www.' in line[i].lower() or '.com' in line[i].lower():
            modifiedLine.append('URL')
        elif '@' in line[i]:
            continue
        else:
            modifiedLine.append(line[i])
            
    line = ' '.join(modifiedLine)
    return line

In [5]:
def no_of_punc_line(line):
    count=0
    punc=['.',',','!','?',':']
    for i in line:
        if i in punc:
            count+=1
    return count

def pos_statistics(pos_text):
    tag_fd = nltk.FreqDist(tag for (word, tag) in pos_text)
    dict_col={'adjectives':0,'punc':1,'adverb':2,'noun':3,'verb':4,'interjection':5,'preposition':6,'article':7,'pronoun':8}
    stats=[0]*len(dict_col)
    for tup in tag_fd.most_common():
        if 'RB' in tup[0]:
            stats[dict_col['adverb']]+=tup[1]
        elif 'NN' in tup[0]:
            stats[dict_col['noun']]+=tup[1]
        elif '.' in tup[0]:
            stats[dict_col['punc']]+=tup[1]
        elif 'JJ' in tup[0]:
            stats[dict_col['adjectives']]+=tup[1]
        elif 'VB' in tup[0]:
            stats[dict_col['verb']]+=tup[1]
        elif 'UH' in tup[0]:
            stats[dict_col['interjection']]+=tup[1]
        elif 'IN' in tup[0]:
            stats[dict_col['preposition']]+=tup[1]
        elif 'PRP' in tup[0] or 'WP' in tup[0]:
            stats[dict_col['pronoun']]+=tup[1]
        elif 'DT' in tup[0]:
            stats[dict_col['article']]+=tup[1]
    return stats

#fet_names=['Total Words']
capitals = string.ascii_uppercase

def f_measure(pos_stats):
	dict_col={'adjectives':0,'punc':1,'adverb':2,'noun':3,'verb':4,'interjection':5,'preposition':6,'article':7,'pronoun':8}
	nouns=pos_stats[dict_col['noun']]
	adjectives=pos_stats[dict_col['adjectives']]
	prep=pos_stats[dict_col['preposition']]
	article=pos_stats[dict_col['article']]
	pro=pos_stats[dict_col['pronoun']]
	verb=pos_stats[dict_col['verb']]
	adv=pos_stats[dict_col['adverb']]
	inter=pos_stats[dict_col['interjection']]
	return 0.5*((nouns+adjectives+prep+article)-(pro+verb+adv+inter)+100)

def count_wrong(text):
	count=0
	for word in text:
		if d1.check(word)==False:
			count+=1
	return count

## More features:

def countHashTags(line):
    p = re.compile('\s#[a-zA-Z0-9]')
    return len(p.findall(line))/len(line.split())

def startsCapital(line):
    if line[0] in capitals:
        return 1
    else:
        return 0

def capitalized(text):
    count = 0
    for word in text:
        if word[0] in capitals:
            try:
                if word[1] not in capitals:
                    count += 1
                else:
                    continue
            except:
                count += 1
    return count/len(text)

def endsWithPeriod(line):
    line = line.split()
    count = 0
    for word in line:
        if word.endswith('.'):
            count += 1
    return (count/len(line))

def totalCapitals(text):
    count = 0
    for word in text:
        if set(word).issubset(set(capitals)):
            count += 1
    return count/len(text)

def perc_count_wrong(text):
	count=0
	for word in text:
		if d1.check(word)==False:
			count+=1
	return count/ len(text)

def perc_of_punc_line(line):
    p = re.compile('[[a-zA-Z][\.!?]\s?$|[a-zA-Z][,:][\sa-zA-Z]')
    count = len(p.findall(line))
    return count/len(line)

def avg_word_len(text):
    chars = 0
    for word in text:
        chars += len(word)
    return chars/len(text)

##############################################################################################
fet_names=['Total Chars','Total Words','Incorrect Words']#'POS_Ones(Variable)'

def words_to_fet():
    file_counter=0
    fp1=open('text.txt','r')
    fp3_a=open('age.txt','r')
    fp4_a=open('gender.txt','r')
    fp2=open('features3.txt','w')
    fp3=open('age_new.txt','w')
    fp4=open('gender_new.txt','w')
    var_count=0
    
    age=[]
    gender=[]
    for line in fp3_a:
        age.append(line)
    for line in fp4_a:
        gender.append(line)
    
    for line in fp1:
        if line=='\n':
            continue
        fet_values=[]
        line=line.replace('\n','')
        
        # For Pre-Processing and handling retweet duplication:
        if preProcessing(line):
            line = preProcessing(line)
        else:
            var_count+=1
            continue
        
        #line = preProcessing(line)
        if file_counter%5000==0:
            print line
            print str(file_counter)
        
        text=nltk.word_tokenize(line)
        pos_text=nltk.pos_tag(text)
        
        # No of Chars
        fet_values.append(len(line))
        
        # No of Words
        fet_values.append(len(pos_text))
        
        # No of wrong words
        #fet_values.append(count_wrong(text))
        fet_values.append(0)
        
        pos_stats=pos_statistics(pos_text)
        
        for i in pos_stats:
            fet_values.append(i)
        
        F_measure=f_measure(pos_stats)
        fet_values.append(F_measure)
        
        # Adding Aditya feats:
        #fet_values.append(startsCapital(line))
        fet_values.append(0)
        fet_values.append(capitalized(text))
        fet_values.append(endsWithPeriod(line))
        fet_values.append(totalCapitals(text))
        fet_values.append(perc_count_wrong(text))
        fet_values.append(perc_of_punc_line(line))
        fet_values.append(avg_word_len(text))
        #fet_values.append(no_of_punc_line(line))
        fet_values.append(0)
        fet_values.append(countHashTags(line))
        
        fp3.write(age[var_count])
        fp4.write(gender[var_count])
        var_count+=1
        for i in range(len(fet_values)):
            fp2.write(str(fet_values[i]))
            if i!=len(fet_values)-1:
                fp2.write(",")
            else:
                fp2.write("\n")
        file_counter+=1            
    fp2.close()

In [6]:
words_to_fet()

Estado civil: aborregao :)) Forges - 9 ENE 2014 URL via
0
The gravitational sucking of rock stardom on the wane? ;)
5000
Day 7 :: Brave #31Days // URL
10000
That Screen In Your Living Room URL via
15000
Multi Estate Extravaganza Online Internet Auction Va URL via
20000
Sobering facts: Climate Reports Forecast Dire Future URL - 375B tons CO2 into atmosphere since Industrial Age
25000


In [14]:
import nltk
from sklearn import svm, cross_validation
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

fet_names=['Total Chars','Total Words','Total Punc']
X_train=[]
Y_gender=[]
Y_age=[]
cols = []

features = ['num_chars','num_words', 'count_wrong','pos_stats',0,0,0,0,0,0,0,0,'f_measure','starts_capital',
                'capitalized_text', 'ends_with','total_caps','perc_count_wrong','perc_punc_line',
                'avg_word_len','num_punc_line', 'hashtag_count'
               ]

sub_features = ['num_chars','num_words', 'pos_stats','f_measure',
                'capitalized_text', 'ends_with','total_caps','perc_count_wrong','perc_punc_line',
                'avg_word_len', 'hashtag_count'
               ]

for feat_name in sub_features:
    if feat_name is not 'pos_stats':
        cols.append(features.index(feat_name))
    else:
        cols.extend([3,4,5,6,7,8,9,10,11])
print cols
fp2=open('gender_new.txt','r')
fp3=open('age_modified.txt','r')
for gender in fp2:
	gender=gender.replace('\n','')
	Y_gender.append(gender)
for age in fp3:
	age=age.replace('\n','')
	Y_age.append(age)
    
# Encoding the labels    
le=preprocessing.LabelEncoder()
le.fit(Y_gender)
print le.classes_
Y_all_gender=le.transform(Y_gender)
le.fit(Y_age)
print le.classes_
Y_all_age=le.transform(Y_age)
X_all=np.loadtxt('features3.txt',delimiter=',', usecols = cols)
print X_all[0]
#X_all=np.loadtxt('features3.txt',delimiter=',')
print Y_all_age.shape
print Y_all_gender.shape
print X_all.shape
#print X_all[0]
split=int(0.8*len(X_all))
X_train=X_all[:split,:]
X_test=X_all[split:,:]
Y_train_gender=Y_all_gender[:split]
Y_actual_gender=Y_all_gender[split:]
Y_train_age=Y_all_age[:split]
Y_actual_age=Y_all_age[split:]

clf1_a = neighbors.KNeighborsClassifier()
#clf.fit(X_train,Y_train)
clf1_a.fit(X_train,Y_train_age)

Y_pred=clf1_a.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "5 nearest neighbor for age: "
print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
#print "Precision: "+str(precision_score(Y_actual_age, Y_pred, average='micro'))
#print "Recall: "+str(recall_score(Y_actual_age, Y_pred, average='micro'))
#print "F1 score: "+str(f1_score(Y_actual_age, Y_pred, average='micro'))

clf1_g = neighbors.KNeighborsClassifier()
#clf.fit(X_train,Y_train)
clf1_g.fit(X_train,Y_train_gender)

Y_pred=clf1_g.predict(X_test)
print "5 nearest neighbor for gender: "
print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
#print "Precision: "+str(precision_score(Y_actual, Y_pred, average='micro'))
#print "Recall: "+str(recall_score(Y_actual, Y_pred, average='micro'))
#print "F1 score: "+str(f1_score(Y_actual, Y_pred, average='micro'))


clf2_g = LogisticRegression()
clf2_g.fit(X_train,Y_train_gender )
Y_pred=clf2_g.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "Logistic Regression for gender: "
print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
#print "Precision: "+str(precision_score(Y_actual, Y_pred, average='micro'))
#print "Recall: "+str(recall_score(Y_actual, Y_pred, average='micro'))
#print "F1 score: "+str(f1_score(Y_actual, Y_pred, average='micro'))
clf2_a = LogisticRegression()
clf2_a.fit(X_train,Y_train_age )
Y_pred=clf2_a.predict(X_test)
print "Logistic Regression for age: "
print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
"""
# LogReg with Cross Validation

Y_pred = cross_validation.cross_val_predict(LogisticRegression(), X_all,Y_all_gender, cv=10)
print "Logistic Regression w/ Cross Validation for gender: "
print "Accuracy: "+str(accuracy_score(Y_all_gender,Y_pred))
Y_pred = cross_validation.cross_val_predict(LogisticRegression(), X_all,Y_all_age, cv=10)
print "Logistic Regression w/ Cross Validation for age: "
print "Accuracy: "+str(accuracy_score(Y_all_age,Y_pred))
"""
clf3_g = svm.SVC()
clf3_g.fit(X_train,Y_train_gender )
Y_pred=clf3_g.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "SVM for gender: "
print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
clf3_a = svm.SVC()
clf3_a.fit(X_train,Y_train_age)
Y_pred=clf3_a.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "SVM for age: "
print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
#print "Precision: "+str(precision_score(Y_actual_age, Y_pred, average='micro'))
#print "Recall: "+str(recall_score(Y_actual_age, Y_pred, average='micro'))
#print "F1 score: "+str(f1_score(Y_actual_age, Y_pred, average='micro'))

"""
clf4_g = GaussianNB()
clf4_g.fit(X_train,Y_train_gender )
Y_pred=clf4_g.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "NB for gender: "
print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
clf4_a=GaussianNB()
clf4_a.fit(X_train,Y_train_age)
Y_pred=clf4_a.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "NB for age: "
print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
"""
clf5_g=DecisionTreeClassifier()
clf5_g.fit(X_train,Y_train_gender )
Y_pred=clf5_g.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "DT for gender: "
print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
clf5_a=DecisionTreeClassifier()
clf5_a.fit(X_train,Y_train_age)
Y_pred=clf5_a.predict(X_test)
#print accuracy_score(Y_actual,Y_pred)
print "DT for age: "
print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))

eclf_a = VotingClassifier(estimators=[('lr', clf2_a), ('knn', clf1_a), ('svc',clf3_a)], voting='hard')
eclf_a.fit(X_train,Y_train_age)
Y_pred=eclf_a.predict(X_test)
print "Ensemble for age: "
print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))

eclf_g = VotingClassifier(estimators=[('lr', clf2_g), ('knn', clf1_g), ('svc',clf3_g)],voting='hard')
eclf_g.fit(X_train,Y_train_gender)
Y_pred=eclf_g.predict(X_test)
print "Ensemble for gender: "
print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))

"""# Ensemble with Cross Validation
eclf_a = VotingClassifier(estimators=[('lr', clf2_a), ('knn', clf1_a), ('svc',clf3_a)], voting='hard')
eclf_g = VotingClassifier(estimators=[('lr', clf2_g), ('knn', clf1_g), ('svc',clf3_g)], voting='hard')
Y_pred = cross_validation.cross_val_predict(eclf_g, X_all,Y_all_gender, cv=10)
print "Ensemble w/ Cross Validation for gender: "
print "Accuracy: "+str(accuracy_score(Y_all_gender,Y_pred))
Y_pred = cross_validation.cross_val_predict(eclf_a, X_all,Y_all_age, cv=10)
print "Ensemble w/ Cross Validation for age: "
print "Accuracy: "+str(accuracy_score(Y_all_age,Y_pred))"""

[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21]
['FEMALE' 'MALE']
['18-24' '23-50' '50-xx']
[  5.50000000e+01   1.40000000e+01   2.00000000e+00   0.00000000e+00
   0.00000000e+00   4.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   0.00000000e+00   0.00000000e+00   5.35000000e+01
   1.42857143e-01   0.00000000e+00   1.42857143e-01   5.00000000e-01
   1.81818182e-02   3.21428571e+00   0.00000000e+00]
(27395,)
(27395,)
(27395, 19)
5 nearest neighbor for age: 
Accuracy: 0.705420697208
5 nearest neighbor for gender: 
Accuracy: 0.612885563059
Logistic Regression for gender: 
Accuracy: 0.565249133053
Logistic Regression for age: 
Accuracy: 0.689906917321
SVM for gender: 
Accuracy: 0.609052746852
SVM for age: 
Accuracy: 0.712903814565
DT for gender: 
Accuracy: 0.65796678226
DT for age: 
Accuracy: 0.662894688812
Ensemble for age: 
Accuracy: 0.712356269392
Ensemble for gender: 
Accuracy: 0.625661617083


'# Ensemble with Cross Validation\neclf_a = VotingClassifier(estimators=[(\'lr\', clf2_a), (\'knn\', clf1_a), (\'svc\',clf3_a)], voting=\'hard\')\neclf_g = VotingClassifier(estimators=[(\'lr\', clf2_g), (\'knn\', clf1_g), (\'svc\',clf3_g)], voting=\'hard\')\nY_pred = cross_validation.cross_val_predict(eclf_g, X_all,Y_all_gender, cv=10)\nprint "Ensemble w/ Cross Validation for gender: "\nprint "Accuracy: "+str(accuracy_score(Y_all_gender,Y_pred))\nY_pred = cross_validation.cross_val_predict(eclf_a, X_all,Y_all_age, cv=10)\nprint "Ensemble w/ Cross Validation for age: "\nprint "Accuracy: "+str(accuracy_score(Y_all_age,Y_pred))'

In [38]:
pwd

u'/Users/aditya/Google Drive/UMass/First Semester/585/Project/WorkArea/Twitter_data/pan14-author-profiling-training-corpus-english-twitter-2014-04-16'

In [74]:
## Finding best accuracies

def computeMaxAccuracy(sub_features):   
    global maxAcc_a
    global maxAcc_g
    global featSet_a
    global featSet_g
    cols = []
    features = ['num_chars','num_words', 'count_wrong','pos_stats',0,0,0,0,0,0,0,0,'f_measure','starts_capital',
                'capitalized_text', 'ends_with','total_caps','perc_count_wrong','perc_punc_line',
                'avg_word_len','num_punc_line','num_hashtags'
               ]
    
    for feat_name in sub_features:
        if feat_name is not 'pos_stats':
            cols.append(features.index(feat_name))
        else:
            cols.extend([3,4,5,6,7,8,9,10,11])
    print cols
    
    X_train=[]
    Y_gender=[]
    Y_age=[]
    
    fp2=open('gender.txt','r')
    fp3=open('age.txt','r')
    for gender in fp2:
        gender=gender.replace('\n','')
        Y_gender.append(gender)
    for age in fp3:
        age=age.replace('\n','')
        Y_age.append(age)

    # Encoding the labels    
    le=preprocessing.LabelEncoder()
    le.fit(Y_gender)
    Y_all_gender=le.transform(Y_gender)
    le.fit(Y_age)
    Y_all_age=le.transform(Y_age)
    X_all=np.loadtxt('features3.txt',delimiter=',', usecols = cols)
    split=int(0.8*len(X_all))
    X_train=X_all[:split,:]
    X_test=X_all[split:,:]
    Y_train_gender=Y_all_gender[:split]
    Y_actual_gender=Y_all_gender[split:]
    Y_train_age=Y_all_age[:split]
    Y_actual_age=Y_all_age[split:]

    clf1_a = neighbors.KNeighborsClassifier()
    clf1_a.fit(X_train,Y_train_age)

    Y_pred=clf1_a.predict(X_test)
    #print accuracy_score(Y_actual,Y_pred)
    #print "5 nearest neighbor for age: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
    acc_Age = accuracy_score(Y_actual_age,Y_pred)
    if acc_Age > maxAcc_a:
        maxAcc_a = acc_Age
        featSet_a = sub_features
    #print "Precision: "+str(precision_score(Y_actual_age, Y_pred, average='micro'))
    #print "Recall: "+str(recall_score(Y_actual_age, Y_pred, average='micro'))
    #print "F1 score: "+str(f1_score(Y_actual_age, Y_pred, average='micro'))

    clf1_g = neighbors.KNeighborsClassifier()
    #clf.fit(X_train,Y_train)
    clf1_g.fit(X_train,Y_train_gender)
    Y_pred=clf1_g.predict(X_test)
    #print "5 nearest neighbor for gender: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
    acc_Gender = accuracy_score(Y_actual_gender,Y_pred)
    if acc_Gender > maxAcc_g:
        maxAcc_g = acc_Gender
        featSet_g = sub_features
    
    #print "Precision: "+str(precision_score(Y_actual, Y_pred, average='micro'))
    #print "Recall: "+str(recall_score(Y_actual, Y_pred, average='micro'))
    #print "F1 score: "+str(f1_score(Y_actual, Y_pred, average='micro'))

    """clf2_g = LogisticRegression()
    #clf.fit(X_train,Y_train)
    clf2_g.fit(X_train,Y_train_gender )
    Y_pred=clf2_g.predict(X_test)
    #print accuracy_score(Y_actual,Y_pred)
    #print "Logistic Regression for gender: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
    acc_Gender = accuracy_score(Y_actual_gender,Y_pred)
    if acc_Gender > maxAcc_g:
        maxAcc_g = acc_Gender
        featSet_g = sub_features
    
    #print "Precision: "+str(precision_score(Y_actual, Y_pred, average='micro'))
    #print "Recall: "+str(recall_score(Y_actual, Y_pred, average='micro'))
    #print "F1 score: "+str(f1_score(Y_actual, Y_pred, average='micro'))
    clf2_a = LogisticRegression()
    #clf.fit(X_train,Y_train)
    clf2_a.fit(X_train,Y_train_age )
    Y_pred=clf2_a.predict(X_test)
    #print "Logistic Regression for age: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
    acc_Age = accuracy_score(Y_actual_age,Y_pred)
    if acc_Age > maxAcc_a:
        maxAcc_a = acc_Age
        featSet_a = sub_features
    """
    # LogReg with Cross Validation

    Y_pred = cross_validation.cross_val_predict(LogisticRegression(), X_all,Y_all_gender, cv=10)
    print "Logistic Regression w/ Cross Validation for gender: "
    print "Accuracy: "+str(accuracy_score(Y_all_gender,Y_pred))
    acc_Gender = accuracy_score(Y_all_gender,Y_pred)
    if acc_Gender > maxAcc_g:
        maxAcc_g = acc_Gender
        featSet_g = sub_features
    
    Y_pred = cross_validation.cross_val_predict(LogisticRegression(), X_all,Y_all_age, cv=10)
    print "Logistic Regression w/ Cross Validation for age: "
    print "Accuracy: "+str(accuracy_score(Y_all_age,Y_pred))
    acc_Age = accuracy_score(Y_all_age,Y_pred)
    if acc_Age > maxAcc_a:
        maxAcc_a = acc_Age
        featSet_a = sub_features

    clf3_g = svm.SVC()
    clf3_g.fit(X_train,Y_train_gender )
    Y_pred=clf3_g.predict(X_test)
    #print accuracy_score(Y_actual,Y_pred)
    #print "SVM for gender: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
    acc_Gender = accuracy_score(Y_actual_gender,Y_pred)
    if acc_Gender > maxAcc_g:
        maxAcc_g = acc_Gender
        featSet_g = sub_features
    
    clf3_a = svm.SVC()
    clf3_a.fit(X_train,Y_train_age)
    Y_pred=clf3_a.predict(X_test)
    #print accuracy_score(Y_actual,Y_pred)
    #print "SVM for age: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
    acc_Age = accuracy_score(Y_actual_age,Y_pred)
    if acc_Age > maxAcc_a:
        maxAcc_a = acc_Age
        featSet_a = sub_features

    #print "Precision: "+str(precision_score(Y_actual_age, Y_pred, average='micro'))
    #print "Recall: "+str(recall_score(Y_actual_age, Y_pred, average='micro'))
    #print "F1 score: "+str(f1_score(Y_actual_age, Y_pred, average='micro'))
"""
    eclf_a = VotingClassifier(estimators=[('lr', clf2_a), ('knn', clf1_a), ('svc',clf3_a)], voting='hard')
    eclf_a.fit(X_train,Y_train_age)
    Y_pred=eclf_a.predict(X_test)
    #print "Ensemble for age: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))
    acc_Age = accuracy_score(Y_actual_age,Y_pred)
    if acc_Age > maxAcc_a:
        maxAcc_a = acc_Age
        featSet_a = sub_features


    eclf_g = VotingClassifier(estimators=[('lr', clf2_g), ('knn', clf1_g), ('svc',clf3_g)],voting='hard')
    eclf_g.fit(X_train,Y_train_gender)
    Y_pred=eclf_g.predict(X_test)
    #print "Ensemble for gender: "
    #print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))
    acc_Gender = accuracy_score(Y_actual_gender,Y_pred)
    if acc_Gender > maxAcc_g:
        maxAcc_g = acc_Gender
        featSet_g = sub_features
        
    # Ensemble with Cross Validation
    eclf_a = VotingClassifier(estimators=[('lr', clf2_a), ('knn', clf1_a), ('svc',clf3_a)], voting='hard')
    eclf_g = VotingClassifier(estimators=[('lr', clf2_g), ('knn', clf1_g), ('svc',clf3_g)], voting='hard')
    Y_pred = cross_validation.cross_val_predict(eclf_g, X_all,Y_all_gender, cv=10)
    #print "Ensemble w/ Cross Validation for gender: "
    #print "Accuracy: "+str(accuracy_score(Y_all_gender,Y_pred))
    acc_Gender = accuracy_score(Y_all_gender,Y_pred)
    if acc_Gender > maxAcc_g:
        maxAcc_g = acc_Gender
        featSet_g = sub_features

    Y_pred = cross_validation.cross_val_predict(eclf_a, X_all,Y_all_age, cv=10)
    #print "Ensemble w/ Cross Validation for age: "
    #print "Accuracy: "+str(accuracy_score(Y_all_age,Y_pred))
    acc_Age = accuracy_score(Y_all_age,Y_pred)
    if acc_Age > maxAcc_a:
        maxAcc_a = acc_Age
        featSet_a = sub_features
"""

'\n    eclf_a = VotingClassifier(estimators=[(\'lr\', clf2_a), (\'knn\', clf1_a), (\'svc\',clf3_a)], voting=\'hard\')\n    eclf_a.fit(X_train,Y_train_age)\n    Y_pred=eclf_a.predict(X_test)\n    #print "Ensemble for age: "\n    #print "Accuracy: "+str(accuracy_score(Y_actual_age,Y_pred))\n    acc_Age = accuracy_score(Y_actual_age,Y_pred)\n    if acc_Age > maxAcc_a:\n        maxAcc_a = acc_Age\n        featSet_a = sub_features\n\n\n    eclf_g = VotingClassifier(estimators=[(\'lr\', clf2_g), (\'knn\', clf1_g), (\'svc\',clf3_g)],voting=\'hard\')\n    eclf_g.fit(X_train,Y_train_gender)\n    Y_pred=eclf_g.predict(X_test)\n    #print "Ensemble for gender: "\n    #print "Accuracy: "+str(accuracy_score(Y_actual_gender,Y_pred))\n    acc_Gender = accuracy_score(Y_actual_gender,Y_pred)\n    if acc_Gender > maxAcc_g:\n        maxAcc_g = acc_Gender\n        featSet_g = sub_features\n        \n    # Ensemble with Cross Validation\n    eclf_a = VotingClassifier(estimators=[(\'lr\', clf2_a), (\'knn\',

In [75]:
from itertools import chain, combinations
maxAcc_a = 0
maxAcc_g = 0
featSet_a = []
featSet_g = []
    
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(3, len(s)+1)))

def createFeatures():
    global maxAcc_a
    global maxAcc_g
    global featSet_a
    global featSet_g
    
    # Uncomment below for creating features for a new dataset!
    #words_to_fet()
    features = ['num_chars','num_words', 'count_wrong','pos_stats','f_measure','starts_capital',
                'capitalized_text', 'ends_with','total_caps','perc_count_wrong','perc_punc_line',
                'avg_word_len','num_punc_line','num_hashtags'
               ]
    count = 0
    for sub_feats in powerset(features)[::-1]:
        count += 1
        if count > 10:
            break
        computeMaxAccuracy(sub_feats)
    return maxAcc_a, maxAcc_g, [i for i in featSet_a], [i for i in featSet_g]

In [76]:
createFeatures()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


ValueError: Found input variables with inconsistent numbers of samples: [5531, 5479]